In [22]:
# 互联网中充斥着大量钓鱼欺诈类网站。这类非法网站通常试图掩人耳目、充当正规网站，而实际上却是在窃取用户的身份、密码、交易等重要信息
# 机器学习在信息安全领域中的一个重要应用就是用来识别这些钓鱼网站。fraudulent.csv在data文件夹中。
# fraudulent.csv文件中含有10,086条数据，每条数据含有18个特征以及1个标签。各个特征的含义如下：
# contain_IP：网址中是否包涵ip，比如http://121.99.3.123/fake.html 包含ip。1表示包含，0表示不包含；
# is_long：网址字符是否过长。1表示网址过长，0表示网址不长；
# is_tinyurl：网址是否是短网址。比如https://bit.ly/2kXX6jV 就是短网址。1表示是短网址，0表示不是；
# contain_at：网址是否包含“@”符号。1表示包含，0表示不包含；
# contain_double_slash：网址是否包含“//”符号，该符号用来表示网址跳转。1表示包含，0表示不包含；
# contain_dash：网址是否包含“-”符号，该符号经常帮助用来伪装真网站，比如www.my-taobao.com 。 1表示包含，0表示不包含；
# contain_subdomain：网址是否包含子域名，比如www.ecnu.edu.cn 就包含edu和cn子域名。1表示包含，0表示不包含；
# is_SSL：网址是否是https安全链接。1表示包含，0表示不包含；
# with_long_history：网址所属的主域名存在的时间。1表示长久，0表示不长久；
# contain_icon：网址网页是否有小图标。1表示包含，0表示不包含；
# contain_ext_domain：该网页是否加载其他域名下的附件或者网页。1表示包含，0表示不包含；
# contain_email_to：该网页是否包含发送邮件的组件。1表示包含，0表示不包含；
# allow_right_click：该网页是否允许用户进行右击操作。1表示允许，0表示不允许；
# contain_pop_up_windowL：该网页是否包含弹窗。1表示包含，0表示不包含；
# contain_Iframe：该网页是否包含Iframe（嵌套网页）。1表示包含，0表示不包含；
# has_DNSRecord：网址是否有DNS记录。1表示有，0表示无；
# traffic：该网站的流量大小。1表示大，0表示小；
# google_rank：该网址在google搜索中的排名。1表示高于同类网站的平均排名，0表示低于同类网站的平均排名；
# y：表示网站是否是钓鱼欺诈网站，1表示是，0表示不是。
# 原始数据中含有大量缺失值，请自行处理这些缺失值（可以剔除缺失值过多的列或者使用众数填充等方法）。
# 将原始数据分为训练集、测试集（随机种子请设置为1）（若有需要可以将训练集进一步分为训练集和验证集）。
# 现在请建立一个二分类模型，使用训练集训练模型，再使用测试集测试模型。
# 评估指标为F1值
# 分类模型可采用：k-近邻、决策树、逻辑回归、支持向量机等。
# 可以与周围同学比较一下F1值的大小（越接近1越好），看看谁的数据预处理和分类模型更强。
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score, recall_score

# 读取数据并查看数据基本信息
data = pd.read_csv('fraudulent.csv')

# 分离特征和标签
X = data.drop('y', axis=1)
y = data['y']

# 根据特征名规律判断分类特征列
true_categorical_cols = [col for col in X.columns if 'contain' in col or 'is_' in col]

# 对于分类特征用众数填充缺失值
cat_imputer = SimpleImputer(strategy='most_frequent')
# 只要列表不为空就进行填充操作
if true_categorical_cols:
    X[true_categorical_cols] = cat_imputer.fit_transform(X[true_categorical_cols])
else:
    print("未找到合适的分类特征列，请检查数据或特征列名规律！")

# 数值特征列
numeric_cols = X.select_dtypes(include=[np.number]).columns

# 对于数值特征，先用均值填充
num_imputer = SimpleImputer(strategy='mean')
X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])

# 特征标准化
scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

# 划分数据集
# 划分训练集和测试集（按8:2划分）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# 从训练集中划分出验证集
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

# 决策树模型超参数调优，添加更多超参数选项以更全面地搜索最优参数组合
param_grid_tree = {
    'max_depth': list(range(3, 10)),
    'min_samples_split': [2, 5, 10],
}
tree_model = DecisionTreeClassifier()
grid_search_tree = GridSearchCV(tree_model, param_grid_tree, cv=5, scoring='f1')
grid_search_tree.fit(X_train, y_train)
best_tree_model = grid_search_tree.best_estimator_

# 模型评估，F1值，准确率和召回率
y_pred = best_tree_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("决策树模型评估结果:")
print(f"F1值: {f1}")
print(f"准确率: {accuracy}")
print(f"召回率: {recall}")

决策树模型评估结果:
F1值: 0.8550955414012739
准确率: 0.9098116947472745
召回率: 0.8430141287284144
